in this notebook we will use dataset that includes all data (some with None value) to build an augmented dataset where None value will be replace with predicted values. 

In [12]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import joblib
import sklearn         

In [13]:
#This dataset includes all data, with and without 'None'

df_ab = pd.read_csv("dataset_CdSe_scaled.csv") 


In [14]:
# Input for ML models

input_col = ['Growth Temp (Celsius)', 'Metal_mmol (mmol)', 'Chalcogen_mmol (mmol)',
             'Amines_mmol (mmol)', 'CA_mmol (mmol)', 'Phosphines_mmol (mmol)', 
             'S_I_amount (g)', 'S_II_amount (g)', 'Time_min (min)', 
             'x0_cadmium acetate', 
             'x0_cadmium acetate dihydrate', 'x0_cadmium oxide', 
             'x0_cadmium stearate', 'x0_dimethylcadmium', 'x1_None', 
             'x1_benzoic acid', 'x1_dodecylphosphonic acid', 
             'x1_ethylphosphonic acid', 'x1_lauric acid', 
             'x1_myrstic acid', 'x1_oleic acid', 'x1_stearic acid',
             'x2_2-6-dimethylpyridine', 'x2_None', 'x2_aniline', 
             'x2_benzylamine', 'x2_dioctylamine/hexadecylamine',
             'x2_dodecylamine', 'x2_heptylamine', 'x2_hexadecylamine', 
             'x2_octadecylamine', 'x2_octylamine', 'x2_oleylamine', 
             'x2_pyridine', 'x2_trioctylamine', 'x3_None', 'x3_diphenylphosphine', 
             'x3_tributylphosphine', 'x3_trioctylphosphine', 
             'x3_triphenylphosphine', 'x4_None', 'x4_liquid parafin', 
             'x4_octadecene', 'x4_phenyl ether', 'x4_trioctylphosphine oxide', 
             'x5_None', 'x5_phosphinic acid', 'x5_trioctylphosphine oxide',
             'diameter_nm']

In [15]:
# Load ML model for predicting absorbance

loaded_rf_ab = joblib.load('model_CdSe_RandomForest_abs_aug.joblib')

# Replace 'None' entries in 'Absorbance max (nm)' column by predicted values.

a = 0
for index, row in df_ab.iterrows():
    if row['abs_nm'] == 'None':
        X = df_ab.loc[index, input_col].to_numpy()
        df_ab.loc[index, 'abs_nm'] = loaded_rf_ab.predict(X.reshape(1, -1))[0]
        a += 1

        
# Save the dataset where all 'None' values in 'Absorbane max (nm)' column are replaced.   
df_ab.to_csv('dataset_CdSe_abs_augmented.csv')

In [16]:
# Now, the dataset only has 'None' values in the 'PL max (nm)' column

df_pl = pd.read_csv('dataset_CdSe_abs_augmented.csv') 

# Load ML model for predicting PL

loaded_rf_pl = joblib.load('model_CdSe_RandomForest_emission_aug.joblib')


# Replace 'None' entries in 'PL max (nm)' column by predicted values.

a = 0
for index, row in df_pl.iterrows():
    if row['emission_nm'] == 'None':
        X = df_pl.loc[index, input_col].to_numpy()
        df_pl.loc[index, 'emission_nm'] = loaded_rf_pl.predict(X.reshape(1, -1))[0]
        a += 1

# Save the dataset where all 'None' values are replaced.
# Final augmented dataset.
# Ready to use for other analysis.

df_pl.to_csv('dataset_CdSe_augmented.csv')